# MungMaster Pro 32x

Script to read in CSV files and export to MySQL.

In [1]:
#Data Ingestion & Manipulation
import pandas as pd
from sqlalchemy import create_engine

#Text manipulation
import re
import csv
import io

#Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

#vector/matrix math
import numpy as np

pd.set_option('max_columns', 500)
plt.style.use('fivethirtyeight')

In [2]:
def importer(path, date_col='Date'):
    '''
    Reads in and formats individual CSV accordingly. Needed only for one-off purposes. 
    '''
    df = pd.read_csv(path)
    df[date_col] = pd.to_datetime(df[date_col])
    
    return df

path = '~/projects/election_money/stacyabramscampaigndec2018.csv'

campaign = importer(path)
campaign.head()

This is a double check to make sure we're getting the output we want. 

In [3]:
def db_import(connection_string):
    '''
    Reads database and appends CSVs row by row.
    '''
    
    

In [6]:
def id_maker(df, col_name, identity_fields, base_number=1000000000):
    '''Read in DataFrame and a list of fields used to identify an entity. Assign a number to those fields.'''
    
    match_table = df[identity_fields].drop_duplicates().copy()
    match_table[col_name] = np.arange(base_number, base_number+len(match_table))
    
    df = pd.merge(df,
                  match_table,
                  on=identity_fields,
                  how='left')
    
    return df[col_name]

In [7]:
def jorbs(df, occupation_col):
    '''Cleans the occupations column'''
    jorbs = df.copy()
    
    jorbs[occupation_col] = jorbs[occupation_col].str.lower() #gets all jobs to the same case for proper comparison
    
    #manual mapping of job names    
    jorb_convert_map = {'not employed': ['not employed', 'retired', 'none', 'unemployed'],
                        'lawyer': ['lawyer', 'attorney', 'lawye', 'general counsel'],
                        'physician': ['doctor', 'pediatrician', 'cardio'],
                        'admin assistant': ['admin asst', 'admin coor', 'administrative assistant'],
                        'communications': ['communication'],
                        'consultant': ['consultant'],
                        'analyst': ['analyst', 'analysis'],
                        'student': ['student'],
                        'real estate': ['real estate'],
                        'education': ['education', 'university', 'teacher', 'professor', 'educator'], 
                        'data professional': ['data'],
                        'engineering': ['engineer', 'programmer', 'engineering'],
                        'sales': ['sales', 'account executive', 'regional vice president', 'business development'],
                        'marketing': ['marketing', 'event', 'social media', 'public relations', 'public affairs'],
                        'arts': ['art', 'design'],
                        'nonprofit': ['nonprofit', 'non profit', 'non-profit'],
                        'nurse': ['nurse', 'nursing'], 
                        'writer': ['writer']}
    
    #uses job mapping to assign titles to common names for better comparison purposes
    for title, jobs in jorb_convert_map.items():
        for jorb in jobs:
            jorbs.loc[jorbs[occupation_col].str.contains(jorb, na=False), occupation_col] = title
    
    return jorbs

In [8]:
def contributor(df, key_col='ContributorId'):
    '''Parse and prep contributor info from the database. Contributor schema:
    
    ContributorId
    LastName
    FirstName
    Address1
    Address2
    City
    State
    Zip
    PAC
    Occupation
    Employer'''
    
    campaign = df.copy() #copy dataframe so as not to impact the data in the original location
    campaign = jorbs(campaign, 'Occupation')
    
    #Prepare dataframe to use same fields as in schema
    contributor_fields = ['ContributorId', 'LastName', 'FirstName', 'Address1', 'Address2',
                          'City', 'State', 'Zip', 'PAC', 'Occupation', 'Employer']
    
    campaign[key_col] = id_maker(df=campaign, 
                                    col_name=key_col, 
                                    identity_fields=['LastName', 'FirstName', 'Address', 'Zip'])
    
    campaign['Address1'] = campaign['Address']
    campaign['Address2'] = np.nan
    
    campaign = campaign[['ContributorId', 'Type', 'LastName', 'FirstName', 'Address', 'City', 'State',
                         'Zip', 'PAC', 'Occupation', 'Employer', 'Date', 'Election', 'Election_Year', 
                         'Cash_Amount', 'In_Kind_Amount', 'In_Kind_Description', 'Candidate_FirstName', 
                         'Candidate_MiddleName', 'Candidate_LastName', 'Candidate_Suffix', 
                         'Committee_Name', 'Address1', 'Address2']]
    
    contributor = campaign[contributor_fields].drop_duplicates(subset=['ContributorId']).reset_index(drop=True)
    
    
    return (contributor, campaign)

In [18]:
#Builds the key to connect with MySQL Database using sqlalchemy and pandas. password can be string of choice, make sure you preserve the punctuation in each string
#file = open('getin.txt')
dialect = ''
driver = 'mysql://'
user = 'root:'
pwd = 'mungmasterpro32x'
host = '@localhost:'
port = '3306'
database = '/ElectionMoney'

key = dialect + driver + user + pwd + host + port + database

In [19]:
#Establish the connection using the key
engine = create_engine(key)
connection = engine.connect()

In [20]:
engine.table_names()

['Candidate',
 'Contribution',
 'Contributor',
 'CSV',
 'Log',
 'Office',
 'Report',
 'ScrapeLog']

In [117]:
def inception(scrape_file='ScrapeLog'):
    '''
    Grabs CSV from SQL database and appends with appropriate metadata'''
    
    #read in the table containing the scraped CSVs
    scrapelog = pd.read_sql(scrape_file, connection)
    
    #initialize a list to store each CSV
    election_master_data = []
    
    #iterate over the table grabbing the CSV for each
    for index, row in scrapelog.iterrows(): 
            
        row_csv_converter = io.StringIO(row[3]) #access the CSV string, which is in position 3 
        reader = csv.reader(row_csv_converter, delimiter=',') #Turn the string into something recognizable as a CSV

        for rows in reader:
            election_master_data.append(rows) #add these into the election master data
            
        #election_master_data.pop(0)    
    
    #define columns for the DataFrame
    cols = ['FilerID', 'Type', 'LastName', 'FirstName', 'Address', 'City',
            'State', 'Zip', 'PAC', 'Occupation', 'Employer', 'Date', 'Election',
            'Election_Year', 'Cash_Amount', 'In_Kind_Amount', 'In_Kind_Description',
            'Candidate_FirstName', 'Candidate_MiddleName', 'Candidate_LastName',
            'Candidate_Suffix', 'Committee_Name', None]
    
    #Turn consumed data into a dataframe
    df = pd.DataFrame(election_master_data)
    df = df.loc[df.loc[:, 0] != 'FilerID'].copy().reset_index(drop=True) #kill the headers that got mixed into the data (if the FilerID is 'FilerID', then it's a header)
    df.columns = cols #set dataframe columns
                
    return df

In [120]:
election_df = inception()

In [127]:
election_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54925 entries, 1 to 56420
Data columns (total 23 columns):
FilerID                 54925 non-null object
Type                    54925 non-null object
LastName                54925 non-null object
FirstName               54925 non-null object
Address                 54925 non-null object
City                    54925 non-null object
State                   54925 non-null object
Zip                     54925 non-null object
PAC                     54925 non-null object
Occupation              54925 non-null object
Employer                54925 non-null object
Date                    54925 non-null object
Election                54925 non-null object
Election_Year           54925 non-null object
Cash_Amount             54925 non-null object
In_Kind_Amount          54925 non-null object
In_Kind_Description     54925 non-null object
Candidate_FirstName     54925 non-null object
Candidate_MiddleName    54925 non-null object
Candidate_LastNam

In [ ]:
contributors.to_sql('Contributor', connection, if_exists='append', index=False) #send the table, appending rows that already exist rather than deleting and replacing

In [47]:
pd.read_sql('Contributor', connection) #readout of the data from the SQL database

,ContributorId,LastName,FirstName,Address1,Address2,City,State,Zip,PAC,Occupation,Employer
0,1000000000,Barlow,Allison,105 Berkeley Pl,None,Brooklyn,NY,11217-3786,None,Program Director,Wallace Global Fund
1,1000000001,Barlow,Cathy,6130 Ardleigh St,None,Philadelphia,PA,19138-1520,None,Attorney,FOCG LLC
2,1000000002,Barlow,Jesse,1427 S Pugh St,None,State College,PA,16801-6132,None,Professor,Penn State
3,1000000003,Barnett,Elizabeth B.,1123 Narcisco St NE,None,Albuquerque,NM,87112-6656,None,Not employed,noneNone
4,1000000004,Barrett,Nolen,346 29th Ave,None,San Francisco,CA,94121-1703,None,Not Employed,Not Employed
5,1000000005,Bartlett,Stephen,659 N Tomahawk Island Dr,None,Portland,OR,97217-7968,None,writer,self
6,1000000006,Bartlett,Steve,659 N Tomahawk Island Dr,None,Portland,OR,97217-7968,None,writer,self employed
7,1000000007,Barwick,William D.,15 Vernon Rd NW,None,Atlanta,GA,30305-2963,None,Partner,Duane Morris LLP
8,1000000008,Baskin,Lisa,PO Box 314,None,Leeds,MA,01053-0314,None,Historian,Self Employed
9,1000000009,Barney,Alicia W,421 E Live Oak St,None,San Gabriel,CA,91776-1519,None,Human Resources,LAUSD
